In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from operator import itemgetter
from pprint import pprint
#import unidecode
import requests

# This notebook is for the extra exercise found in chapter 1
### Data Exercies 4 - Collecting data on a specific car model and describe the data collection method

#### I will scrape hasznaltauto.hu - the biggest Hungarian used car website, the model of my choice is: Renault Megane

In [3]:
#On the site, the only way to see cars is to filter for the relevant population
#The URL leads to a site that is already filtered for Reanult Megane models 
URL = 'https://www.hasznaltauto.hu/talalatilista/PDNG2VG3R3NDAEH5C57QCSLAO5VR4W5NRJKKDFJK6U2RVSAQRQXYG3BHLJAPY66HJFFJGLCPBTT5QM3TM2HAHMTYSG3SBM24BIGRC27SBK3NRCRVMOMRJYMKC3GQLGSIDW2JIAWM6TQHNQEPZJLRQMNUKBA3RCXVJYXIMJBNYMDHY2MVJNERMUUIHEBZ6Y3GEUCTNPU43TNT4I6SWVDX3LXKYDCPEBG6GZOR4KNS533VLCQ2HS4L2AVDYKJQG6KSBKDNYE4ONPXNBIZZMKSCP5CDHG64GKOCBGDI4WGLQGN6IWLMFINLB3EBRU2I3QL7PK4GDD5BGEY2IGW4LW7ORCV5ZXD7YPPYYP75T4AIP2JMLX7ILO24PUATHEL5QNZBSL62KQX46BSCSKTSTOVKOX52NN7TNW2BKVLYR22Y6ANEUVNFX4CYXTMWFH5MMBCVNDGMBMH2VTEIXYKOKTTSNHGQZS3IYLMPBXRJH7J7TDE74WN5K2WYVU2EHSCQHWZQIZW6PGJG2QSBL563WCC4CXNATZLAAY7AJRI2YQNO4RG7KZXNS2VGJVWHPGFPGPROKJOPJ4JSYXFT4XUJ6WOQHHWQCFOY3PXRL52E5ONJ2M4QHO4GOTE7NMZQ2PKBQDHYSVRE2PDDQP3EUOVVZ5DW7XPY2MV3KEDBXRET4TBBLQ2TWWEPOFPLWCTUKCPWGMDDCQG6VGCMCJFNQGOQGNNYA2OYVVHM3YEI62OGUT5SOWXSU6PRVLEV3WRRY4Z4C4E6GXNJFIZDPGVMP6DL76K6BVZ3I3M6ZWRIHOIZVHRZK5BXPQKQD3ENOYHMVDHXWP3OG35Q7WRXC3O2RWP3L6E2DDNR'

final_cars_df = pd.DataFrame(columns = ['ID', 'ShortDesc',  'Prices', 'Engine', 'Year'
                                       , 'Displacement','Power_kW', 'Horsepower', 'Km','PictureCount'])

In [5]:
#Getting URLs - each of them needs to be scraped separately, one page has only 20 cars available 
response = requests.get(URL)
soup = BeautifulSoup(response.content, "html.parser") 

#Get URL base from button that helps changing pages
url_list = soup.find('ul',{'class':'pagination'}).findAll('a')
url_base = [elem.get('href') for elem in url_list]
url_base = url_base[0].split('page')[0]

#getting the number of the last page
max_page = soup.findAll('li', {'class':'last'})
max_page = [elem.getText() for elem in max_page]
max_page = max_page[0]

#Create URL Links to go over
url_pages = [url_base+'page'+str(elem) for elem in range(int(max_page)+1)[1:]]

In [50]:
def hasznaltauto_scraper(URL):    
    
    global final_cars_df
    
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, "html.parser") 
    
    id_list = soup.find('div',{'id' : 'w27'}).findAll('div',{'class':'talalatisor-info talalatisor-hirkod'})
    id_list = [elem.getText().replace('(Hirdetéskód: ','').replace(')','') for elem in id_list]

    short_desc = soup.find('div',{'id' : 'w27'}).findAll('h3')
    short_desc = [elem.getText() for elem in short_desc]

    prices = soup.find('div',{'id' : 'w27'}).findAll('div',{'class':'col-sm-9 hidden-xs'})
    prices = [elem.getText().replace('\xa0','') for elem in prices]

    infos = soup.findAll('div',{'class':'talalatisor-info adatok'})
    infos = [elem.getText() for elem in infos]

    infos_temp = []
    for elem in infos:       
        elem_list = elem.split(', ')
        if len(elem_list) != 6:
            elem_list = 6*[np.NaN]
        infos_temp.append(elem_list)

    engine = [elem[0] for elem in infos_temp]
    year = [elem[1] for elem in infos_temp]
    displacement = [str(elem[2]).replace('\xa0','').replace(' cm³','') for elem in infos_temp]
    kw_power = [str(elem[3]).replace(' kW','') for elem in infos_temp]
    horsepower = [str(elem[4]).replace(' LE','') for elem in infos_temp]
    km = [str(elem[5]).replace('\xa0','').replace('km','') for elem in infos_temp]

    pic_count = soup.findAll('div',{'class':'col-xs-7'})
    pic_count = [elem.getText() for elem in pic_count]
    pic_count = pic_count[::3]

    dict_for_cars = {'ID': id_list, 'ShortDesc':short_desc,  'Prices': prices
                    , 'Engine' : engine, 'Year': year, 'Displacement': displacement
                    ,'Power_kW': kw_power, 'Horsepower' : horsepower,'Km': km, 'PictureCount':pic_count}


    cars_df = pd.DataFrame(data= dict_for_cars,columns = ['ID', 'ShortDesc',  'Prices', 'Engine', 'Year'
                                           , 'Displacement','Power_kW', 'Horsepower', 'Km','PictureCount'])
    final_cars_df = final_cars_df.append(cars_df,ignore_index = True)


In [52]:
final_cars_df = pd.DataFrame(columns = ['ID', 'ShortDesc',  'Prices', 'Engine', 'Year'
                                       , 'Displacement','Power_kW', 'Horsepower', 'Km','PictureCount'])

In [53]:
for link in url_pages:
    hasznaltauto_scraper(link)
    print(link + ' is ready')

https://www.hasznaltauto.hu/talalatilista/PDNG2VG3R3NDAEH5C57QCSOYWI2Y7LKWIWVFBJKKPWGQMMQE4PFSBW4JCYIP7XTRIK5XDFRHQZZTYM3HY3DQCWNN4QW4RIUUIJB4JFV4QIW7MYWNLAQULOFCIVZYCLWSAAW2KAETD7ODF7RKYWHHUFJRYYIUROJ5IY2OA25LLTFF2SBB4YGLYT4ZC4U3B43VS23X2RVEWTWUHTQRYLKYDCPVBG6O3BR2KIKMH32KRILDZOF5AKR4FEYDMUNBADFZE4OPPXFBI5Z4ISCP5DKFT6MGCWCR4GWWEDLDXOOIDB5IGU4ECM6LJ4UUA6LM6ZZXBV6WZ2MBJWTJ3AX7E3SVCPEGZ3CJBOVT5ZZDDSDYTXF7QFX44FRWXPE4D5SPCN72L3WTDDCEZEG7MXMIMR72UED7PGZBIFNZJUZ5EX4GWQ7ZX3LD32QRD55R4A25JKSJP4VV4Q5RJTIVOJVIIFRR4DCPL5KGL5OIRHVNZBITTLM72Y6PBOC7RRAZ76M4YGJ4XVLMSXSZEHPMFAL5DGR3N42KDGKZKYV5DV5RVXAF3K6AGGADM5RP7AT4SE3TKXXGODOUUFTDLSTLQHHDFLC5KGBPKO6BZYYXWG7HHQA3HKRGXULX73JZYOMXX4A25QFOEFO3GMJNZRTOUDAG7RGPCTF4MOB7MSR2ZHT2HN3B7XU5SWUIGC6USPSMEFODK623J5Y345QV5CQN5J3AYEUBVVBRTESNJMBTUBT3MAGTWFNZ3GWBCHWTT5OOVIG66LLZ6GVMSXP2FHDTXQLQTYE63SORSE6LKU34GV7D76C5J5ITNOZWZJTKHJQPAB3UG56BKAP4RN3A5RCOO7K4G6N72O7NHOFW4IXW7YLZHL4TJI/page1 is ready
https://www.hasznaltauto.hu/talalatilista/PDNG2VG3R3NDAEH5C57QCSOYWI

In [55]:
final_cars_df= final_cars_df.set_index('ID')

In [56]:
final_cars_df.describe()

,ShortDesc,Prices,Engine,Year,Displacement,Power_kW,Horsepower,Km,PictureCount
count,1150,1150,1127,1127,1150,1150,1150,1150,1150
unique,744,494,4,287,20,39,39,799,13
top,RENAULT MEGANE 1.4 Authentique,699000Ft,Benzin,2016/5,1461,81,110,nan,6
freq,16,26,563,15,418,184,184,23,421


In [58]:
final_cars_df.to_csv(r'C:\Users\T450s\Python_directory\used_cars.csv', 
                          index = True, sep=',', encoding='utf-8')

In [3]:
used_cars_df = pd.read_csv(r'C:\Users\T450s\Python_directory\used_cars.csv', index_col=0)

In [14]:
used_cars_df.groupby(['Engine']).count()

,ShortDesc,Prices,Year,Displacement,Power_kW,Horsepower,Km,PictureCount
Engine,,,,,,,,
Benzin,563,563,563,563,563,563,563,563
Benzin/Gáz,7,7,7,7,7,7,7,7
Dízel,553,553,553,553,553,553,553,553
LPG,4,4,4,4,4,4,4,4
